In [33]:
import pandas as pd
import lightgbm as lgb
from sklearn import preprocessing
from itertools import product

In [34]:
demanda=pd.read_csv("D:\\Data Science Game 2017 Finals\\demand_anonymized_20170802.csv",sep=";")
evaluation=pd.read_csv("D:\\Data Science Game 2017 Finals\\eval_correct.csv")
sample_submission=pd.read_csv("D:\\Data Science Game 2017 Finals\\sample_submission.csv")

In [35]:
demanda["LogABC"].value_counts()

Nc4e    3462055
hfuf    1360032
XRJD     752676
Name: LogABC, dtype: int64

In [36]:
demanda["MktABC"].value_counts()

9rfJ    1907808
vegq    1596658
OG0i    1141219
ARa9     874703
IttB      75313
Name: MktABC, dtype: int64

In [37]:

del demanda["ID"]

demanda.isnull().sum()

First_MAD                       0
SalOrg                          0
DC                              0
Ship_To                         0
ordre                           0
Plant                           0
Material                        0
ItemCat                         0
OrderQty                        0
LT                              1
LogABC                      20938
MOQ                             1
ROP                             1
SafetyStk                       1
PL                              0
MktABC                          0
SubFct                          0
Gross_Weight                    0
Length                          0
Width                           0
Height                          0
Volume                          0
Gamma                           0
Manufacturer                    0
Business                        0
Month                           0
CBO_CBO_Qty_Shortage      2871107
Age_ZN_ZI_years           3741938
DP_FAMILY_CODE            1952528
PRODUCT_STATUS

In [38]:
evaluation["date"].unique()

array(['2017-04', '2017-05', '2017-06'], dtype=object)

In [39]:
len(demanda["Month"].unique())

63

In [40]:
demanda["Material"].unique()

array(['iR1zr2', 'mD2b7j', 'AjywTn', ..., 'NI55wg', 'oPLnST', 'ja53gA'], dtype=object)

In [41]:
len(demanda["Material"].unique())

14670

In [42]:
demanda["SalOrg"].unique()

array(['yqSu', 'U12J', 'OQfZ', '97LK'], dtype=object)

In [43]:
len(demanda["SalOrg"].unique())

4

In [44]:
evaluation.tail(5)#demanda["Month"].unique(),demanda["Material"].unique(),demanda["SalOrg"].unique()

,ID,SalOrg,Material,date
116023,116023,yqSu,Zzmafz,2017-06
116024,116024,yqSu,zzOfA5,2017-06
116025,116025,yqSu,ZZu2jg,2017-06
116026,116026,yqSu,ZZuJB2,2017-06
116027,116027,yqSu,zzYnFP,2017-06


In [45]:
len(evaluation)

116028

In [46]:
demanda.head(4)

,First_MAD,SalOrg,DC,Ship_To,ordre,Plant,Material,ItemCat,OrderQty,LT,...,ORIGINAL_SUPPLIER,SUBRANGE,Comp_reference_number,Name_Of_Competitor,COMP_PRICE_MIN,COMP_PRICE_AVG,COMP_PRICE_MAX,PRICE,NEAREST_COMP_PRICE_MIN,NEAREST_COMP_PRICE_MAX
0,2014-09-12,yqSu,TUiR,MsHvUF,ggT8Bq,vP6T,iR1zr2,lvOi,1,28.0,...,X6qzbY,qmsY,1.0,1.0,136.800,136.800000,136.80,152.95,136.80,136.80
1,2016-02-12,yqSu,csdY,BA5kLC,AIZGVy,vP6T,mD2b7j,lvOi,10,35.0,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-05-24,U12J,TUiR,MBZhLQ,8Z4g83,Cisr,AjywTn,lvOi,2,28.0,...,SXlc2j,bUkB,3.0,2.0,411.939,677.394667,830.28,908.60,830.28,830.28
3,2012-07-12,yqSu,csdY,9Co4iE,RNuiEu,vP6T,DPiXfc,lvOi,10,35.0,...,Rmt5iw,5cy5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
demanda.columns

Index(['First_MAD', 'SalOrg', 'DC', 'Ship_To', 'ordre', 'Plant', 'Material',
       'ItemCat', 'OrderQty', 'LT', 'LogABC', 'MOQ', 'ROP', 'SafetyStk', 'PL',
       'MktABC', 'SubFct', 'Gross_Weight', 'Length', 'Width', 'Height',
       'Volume', 'Gamma', 'Manufacturer', 'Business', 'Month',
       'CBO_CBO_Qty_Shortage', 'Age_ZN_ZI_years', 'DP_FAMILY_CODE',
       'PRODUCT_STATUS', 'ORIGINAL_SUPPLIER', 'SUBRANGE',
       'Comp_reference_number', 'Name_Of_Competitor', 'COMP_PRICE_MIN',
       'COMP_PRICE_AVG', 'COMP_PRICE_MAX', 'PRICE', 'NEAREST_COMP_PRICE_MIN',
       'NEAREST_COMP_PRICE_MAX'],
      dtype='object')

In [48]:
demanda["MOQ"].describe()

count    5.595700e+06
mean     1.403106e+02
std      2.398007e+02
min      0.000000e+00
25%      7.000000e+00
50%      2.700000e+01
75%      1.750000e+02
max      4.800000e+03
Name: MOQ, dtype: float64

In [49]:
demanda["PL"].value_counts()

ss6l    1736241
c2Z5    1548683
PLsu    1077032
NmYB     730975
9CJ3     502770
Name: PL, dtype: int64

In [50]:
demanda["Business"].value_counts()

32af    2500936
otZd    1543726
JLqu    1534413
deRY      11684
C0Gw       3316
Achw       1626
Name: Business, dtype: int64

In [51]:
demanda["Comp_reference_number"].describe()

count    5.258191e+06
mean     5.777209e-01
std      8.493506e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      9.000000e+00
Name: Comp_reference_number, dtype: float64

In [52]:
## recordar que los acumulados sirven para numericas

#demanda.groupby(['SalOrg','Material',"Year","Month"]).agg({["OrderQty"]:"sum",["LT","MOQ","ROP","SafetyStk","Age_ZN_ZI_years","Comp_reference_number","Name_Of_Competitor","PRICE","NEAREST_COMP_PRICE_MIN","NEAREST_COMP_PRICE_MAX"]:"mean",["Gross_Weight","Length","Width","Height","Volume","COMP_PRICE_MIN","COMP_PRICE_AVG","COMP_PRICE_MAX","COMP_PRICE_MIN"]:"first"})

#demanda_c=demanda.groupby(['SalOrg','Material'"Month"])["LT","MOQ","ROP","SafetyStk","Age_ZN_ZI_years","Comp_reference_number","Name_Of_Competitor","PRICE","NEAREST_COMP_PRICE_MIN","NEAREST_COMP_PRICE_MAX"].mean().reset_index()

a=["SafetyStk","LT","MOQ","ROP","Age_ZN_ZI_years","Comp_reference_number","Name_Of_Competitor","PRICE","NEAREST_COMP_PRICE_MIN","NEAREST_COMP_PRICE_MAX"]

demanda_c=demanda[["CBO_CBO_Qty_Shortage","SafetyStk","LT","MOQ","ROP","Age_ZN_ZI_years","Comp_reference_number","Name_Of_Competitor","PRICE","NEAREST_COMP_PRICE_MIN","NEAREST_COMP_PRICE_MAX",'SalOrg','Material',"Month"]].groupby(['SalOrg','Material',"Month"]).mean().reset_index()

b=["Gross_Weight","Length","Width","Height","Volume","COMP_PRICE_MIN","COMP_PRICE_AVG","COMP_PRICE_MAX","COMP_PRICE_MIN"]

#demanda["SafetyStk_std"]=demanda["SafetyStk"]
#demanda_c=demanda_c.merge(demanda[["SafetyStk_std",'SalOrg','Material',"Month"]].groupby(['SalOrg','Material',"Month"]).std().reset_index(),on=['SalOrg','Material',"Month"],how="left")

demanda_c=demanda_c.merge(demanda[["Gross_Weight","Length","Width","Height","Volume","COMP_PRICE_MIN","COMP_PRICE_AVG","COMP_PRICE_MAX","COMP_PRICE_MIN",'SalOrg','Material',"Month"]].groupby(['SalOrg','Material',"Month"]).first().reset_index(),on=['SalOrg','Material',"Month"],how="left")

c=a+b

#demanda_c=demanda_c.merge(demanda[["DC","LogABC","MktABC","Gamma","Business","PRODUCT_STATUS","ORIGINAL_SUPPLIER","SUBRANGE",'SalOrg','Material',"Month"]].groupby(['SalOrg','Material',"Month"]).agg(lambda x:x.value_counts().index[0]),how="left",on=['SalOrg','Material',"Month"])



evaluation["Month"]=evaluation["date"]

evaluation["Month"]=pd.to_datetime(evaluation["Month"],format="%Y/%m")





#### al final, ensamblar con los ultimos 6 meses

In [53]:
for x in ["DC","LogABC","MktABC","Gamma","Manufacturer","Business","PRODUCT_STATUS","ORIGINAL_SUPPLIER","SUBRANGE"]:
    

    
    demanda_c=demanda_c.merge(demanda.groupby(['SalOrg',"Month"])[x].apply(pd.Series.mode).reset_index(),how="left",on=['SalOrg',"Month"])
    
    del demanda_c["level_2"]
    
    print (x)

c.extend(["DC","LogABC","MktABC","Gamma","Manufacturer","Business","PRODUCT_STATUS","ORIGINAL_SUPPLIER","SUBRANGE"])

DC
LogABC
MktABC
Gamma
Manufacturer
Business
PRODUCT_STATUS
ORIGINAL_SUPPLIER
SUBRANGE


In [54]:
evaluation["Month"]=evaluation["Month"].apply(lambda x: x.strftime('%Y-%m'))

In [55]:

demanda_c=demanda_c.merge(evaluation,on=["SalOrg","Material","Month"],how="outer")

demanda_c.sort_values(['SalOrg','Material',"Month"],inplace=True)


## estos son las ultimas 3 veces, donde se le encontro data
#for x in c:

#    try:
#        demanda_c.loc[:,x+"_3"]=demanda_c.groupby(['SalOrg','Material'])[x].shift(3)
#        demanda_c.loc[:,x+"_4"]=demanda_c.groupby(['SalOrg','Material'])[x].shift(4)
#        demanda_c.loc[:,x+"_5"]=demanda_c.groupby(['SalOrg','Material'])[x].shift(5)


        
#    except:
#        pass

In [56]:
demanda_c=demanda_c.merge(demanda.groupby(['SalOrg','Material',"Month"])['OrderQty'].sum().reset_index(),on=['SalOrg',"Material","Month"],how="outer")

In [57]:
demanda_c["Month"].unique()

array(['2012-05', '2012-06', '2012-07', '2012-09', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07',
       '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', '2016-01',
       '2016-02', '2016-03', '2016-05', '2016-06', '2016-07', '2016-08',
       '2016-09', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03',
       '2017-04', '2017-05', '2017-06', '2012-02', '2012-03', '2012-04',
       '2012-01', '2016-04', '2016-10', '2012-08', '2012-10', '2015-02'], dtype=object)

In [58]:
demanda_c.columns

Index(['SalOrg', 'Material', 'Month', 'CBO_CBO_Qty_Shortage', 'SafetyStk',
       'LT', 'MOQ', 'ROP', 'Age_ZN_ZI_years', 'Comp_reference_number',
       'Name_Of_Competitor', 'PRICE', 'NEAREST_COMP_PRICE_MIN',
       'NEAREST_COMP_PRICE_MAX', 'Gross_Weight', 'Length', 'Width', 'Height',
       'Volume', 'COMP_PRICE_MIN', 'COMP_PRICE_AVG', 'COMP_PRICE_MAX',
       'COMP_PRICE_MIN', 'DC', 'LogABC', 'MktABC', 'Gamma', 'Manufacturer',
       'Business', 'PRODUCT_STATUS', 'ORIGINAL_SUPPLIER', 'SUBRANGE', 'ID',
       'date', 'OrderQty'],
      dtype='object')

In [59]:
evaluation["date"].unique()

array(['2017-04', '2017-05', '2017-06'], dtype=object)

In [60]:
month=demanda["Month"].unique().tolist()
#month.extend(['2017-04', '2017-05', '2017-06'])

x={"Month":month,"Material":demanda["Material"].unique().tolist(),"SalOrg":demanda["SalOrg"].unique().tolist()}


train=pd.DataFrame(list(product(*x.values())), columns=x.keys())

In [61]:
train=train.merge(demanda_c,on=["Month","Material","SalOrg"],how="outer")

In [62]:
del demanda_c

In [63]:
train.loc[train["OrderQty"].isnull(),"OrderQty"]=0

In [64]:
train["Month_ord"]=pd.to_datetime(train["Month"],yearfirst=True).dt.year*100+pd.to_datetime(train["Month"],yearfirst=True).dt.month

In [65]:
train.sort_values(['SalOrg','Material',"Month_ord"],ascending=True,inplace=True)

In [66]:
#for x in train.columns[train.dtypes != object]:
#    train[x]=train[x].fillna(train[x].mean())

for x in c:
    #train.loc[train.loc[,b].isnull(),["Material","SalOrg",b]]=
    train[x]=train.groupby(["Material","SalOrg"])[x].fillna(method="ffill")
    print (str(x)+"-forward-both")
    
for x in c:
    #train.loc[train.loc[,b].isnull(),["Material","SalOrg",b]]=
    train[x]=train.groupby(["Material","SalOrg"])[x].fillna(method="bfill")
    print (str(x) + "-backward-both")
    
for x in c:
    #train.loc[train.loc[,b].isnull(),["Material","SalOrg",b]]=
    train[x]=train.groupby(["Material"])[x].fillna(method="ffill")
    print (str(x)+"-forward-mat")
    
for x in c:
    #train.loc[train.loc[,b].isnull(),["Material","SalOrg",b]]=
    train[x]=train.groupby(["Material"])[x].fillna(method="bfill")
    print (str(x) + "-backward-mat")
    
for x in c:
    #train.loc[train.loc[,b].isnull(),["Material","SalOrg",b]]=
    train[x]=train.groupby(["SalOrg"])[x].fillna(method="ffill")
    print (str(x)+"-forward-sal")
    
for x in c:
    #train.loc[train.loc[,b].isnull(),["Material","SalOrg",b]]=
    train[x]=train.groupby(["SalOrg"])[x].fillna(method="bfill")
    print (str(x) + "-backward-sal")


SafetyStk-forward-both
LT-forward-both
MOQ-forward-both
ROP-forward-both
Age_ZN_ZI_years-forward-both
Comp_reference_number-forward-both
Name_Of_Competitor-forward-both
PRICE-forward-both
NEAREST_COMP_PRICE_MIN-forward-both
NEAREST_COMP_PRICE_MAX-forward-both
Gross_Weight-forward-both
Length-forward-both
Width-forward-both
Height-forward-both
Volume-forward-both
COMP_PRICE_MIN-forward-both
COMP_PRICE_AVG-forward-both
COMP_PRICE_MAX-forward-both
COMP_PRICE_MIN-forward-both
DC-forward-both
LogABC-forward-both
MktABC-forward-both
Gamma-forward-both
Manufacturer-forward-both
Business-forward-both
PRODUCT_STATUS-forward-both
ORIGINAL_SUPPLIER-forward-both
SUBRANGE-forward-both
SafetyStk-backward-both
LT-backward-both
MOQ-backward-both
ROP-backward-both
Age_ZN_ZI_years-backward-both
Comp_reference_number-backward-both
Name_Of_Competitor-backward-both
PRICE-backward-both
NEAREST_COMP_PRICE_MIN-backward-both
NEAREST_COMP_PRICE_MAX-backward-both
Gross_Weight-backward-both
Length-backward-both
W

In [67]:
for x in c:

    try:
        train.loc[:,x+"_3"]=train.groupby(['SalOrg','Material'])[x].shift(3)
        #train.loc[:,x+"_4"]=train.groupby(['SalOrg','Material'])[x].shift(4)
        #train.loc[:,x+"_5"]=train.groupby(['SalOrg','Material'])[x].shift(5)
        
        



        
    except:
        pass
    
train.loc[:,"OrderQty_3"]=train.groupby(['SalOrg','Material'])["OrderQty"].shift(3)
train.loc[:,"OrderQty_4"]=train.groupby(['SalOrg','Material'])["OrderQty"].shift(4)
train.loc[:,"OrderQty_5"]=train.groupby(['SalOrg','Material'])["OrderQty"].shift(5)
train.loc[:,"OrderQty_6"]=train.groupby(['SalOrg','Material'])["OrderQty"].shift(6)
train.loc[:,"OrderQty_12"]=train.groupby(['SalOrg','Material'])["OrderQty"].shift(12)

train.loc[:,"OrderQty_3_S"]=train.groupby(['SalOrg'])["OrderQty"].shift(3)
train.loc[:,"OrderQty_4_S"]=train.groupby(['SalOrg'])["OrderQty"].shift(4)
train.loc[:,"OrderQty_5_S"]=train.groupby(['SalOrg'])["OrderQty"].shift(5)
train.loc[:,"OrderQty_6_S"]=train.groupby(['SalOrg'])["OrderQty"].shift(6)
train.loc[:,"OrderQty_12_S"]=train.groupby(['SalOrg'])["OrderQty"].shift(12)

train.loc[:,"OrderQty_3_M"]=train.groupby(["Material"])["OrderQty"].shift(3)
train.loc[:,"OrderQty_4_M"]=train.groupby(["Material"])["OrderQty"].shift(4)
train.loc[:,"OrderQty_5_M"]=train.groupby(["Material"])["OrderQty"].shift(5)
train.loc[:,"OrderQty_6_M"]=train.groupby(["Material"])["OrderQty"].shift(6)
train.loc[:,"OrderQty_12_M"]=train.groupby(['Material'])["OrderQty"].shift(12)



In [68]:
train["OrderQty_fr"]=(train.loc[:,"OrderQty_3"]+train.loc[:,"OrderQty_4"]+train.loc[:,"OrderQty_5"]+train.loc[:,"OrderQty_6"])/4

train["OrderQty_fr_12"]=train["OrderQty_fr"]/train["OrderQty_12"]

train["OrderQty_change"]=train["OrderQty_fr_12"]-train["OrderQty_fr"]

In [69]:
del demanda

In [70]:
#for x in train.columns[train.dtypes != object]:
#    train[x]=train[x].fillna(train[x].mean())



In [71]:
#pd.to_datetime(train["Month"],yearfirst=True)

#train["Month"].map(lambda x: x.strftime('%Y-%m-%d'))

train["Month"]=pd.to_datetime(train["Month"]).map(lambda x: x.strftime('%Y-%m'))

train["Mes"]=pd.to_datetime(train["Month"],yearfirst=True).dt.month

train["Mes_cat"]=train["Mes"].astype(str)




In [72]:
def suma_acumulada(unidad,var_numerica,data):
    ent=unidad+var_numerica
    #data.sort_values(ent,inplace=True)
    data.reset_index(drop=True,inplace=True)
    return data[ent].groupby(unidad).expanding(min_periods=3).sum().reset_index(drop=True)[var_numerica]

def promedio_acumulado(unidad,var_numerica,data):
    ent=unidad+var_numerica
    #data.sort_values(ent,inplace=True)
    data.reset_index(drop=True,inplace=True)
    return data[ent].groupby(unidad).expanding(min_periods=3).mean().reset_index(drop=True)[var_numerica]

def rolling_promedio(unidad,var_numerica,data,periodo):
    ent=unidad+var_numerica
    #data.sort_values(ent,inplace=True)
    data.reset_index(drop=True,inplace=True)
    return data[ent].groupby(unidad).rolling(min_periods=3,window=periodo).mean().reset_index(drop=True)[var_numerica]

In [73]:
#train["SafetyStk_3",'MOQ_3','ROP_3','Volume_3','Length_3','Width_3']

train[["SafetyStk_3_3_MESES","MOQ_3_3_MESES","ROP_3_3_MESES",'Volume_3_3_MESES',"Length_3_3_MESES","Width_3_3_MESES"]]=rolling_promedio(["SalOrg","Material"],["SafetyStk_3",'MOQ_3','ROP_3','Volume_3','Length_3','Width_3'],train,3)
#vida_t[["INGRESO_ACUMULADO","SALDO_TC_SBS_ACUMULADO","PRIMA_ACUMULADO","NUM_PRODUCTOS_ACUMULADO","CAMBIO_PRODUCTO_ACUMULADO","MONTO_DOLARES_ACUMULADO","MORA_ACUMULADO","NUM_FACTURAS_ACUMULADO","CAMBIO_RIESGO_ACUMULADO","NUM_RIESGOS_ACUMULADO",'SEGMENTO_INTERNO_NUM_ACUMULADO',"CAPITAL_ASEGURADO_ACUMULADO","NUM_RAMOPROP_ACUMULADO",'SALDO_SBS_ACUMULADO']]=promedio_acumulado(["CUC"],['INGRESO','SALDO_TC_SBS','PRIMA',"NUM_PRODUCTOS","CAMBIO_PRODUCTO","MONTO_DOLARES","MORA","NUM_FACTURAS","CAMBIO_RIESGO","NUM_RIESGOS",'SEGMENTO_INTERNO',"CAPITAL_ASEGURADO","NUM_RAMOPROP",'SALDO_SBS'],["PERIODO"],vida_t)

train[["SafetyStk_3_ACUM","MOQ_ACUM_ACUM","ROP_ACUM_ACUM",'Volume_3_ACUM',"Length_3_ACUM","Width_ACUM"]]=promedio_acumulado(["SalOrg","Material"],["SafetyStk_3",'MOQ_3','ROP_3','Volume_3','Length_3','Width_3'],train)

#vida_t["INGRESO_3_MESES_DIFERENCIA"]=vida_t["INGRESO_3_MESES"]-vida_t["INGRESO"]
#vida_t["INGRESO_ACUMULADO_DIFERENCIA"]=vida_t["INGRESO_ACUMULADO"]-vida_t["INGRESO"]

#vida_t["NUM_RAMOPROP_3_MESES_DIFERENCIA"]=vida_t["NUM_RAMOPROP_3_MESES"]-vida_t["NUM_RAMOPROP"]
#vida_t["NUM_RAMOPROP_ACUMULADO_DIFERENCIA"]=vida_t["NUM_RAMOPROP_ACUMULADO"]-vida_t["NUM_RAMOPROP"]

In [74]:
train.drop(c, axis=1, inplace=True)

train.columns

Index(['Month', 'Material', 'SalOrg', 'CBO_CBO_Qty_Shortage', 'ID', 'date',
       'OrderQty', 'Month_ord', 'SafetyStk_3', 'LT_3', 'MOQ_3', 'ROP_3',
       'Age_ZN_ZI_years_3', 'Comp_reference_number_3', 'Name_Of_Competitor_3',
       'PRICE_3', 'NEAREST_COMP_PRICE_MIN_3', 'NEAREST_COMP_PRICE_MAX_3',
       'Gross_Weight_3', 'Length_3', 'Width_3', 'Height_3', 'Volume_3',
       'COMP_PRICE_AVG_3', 'COMP_PRICE_MAX_3', 'DC_3', 'LogABC_3', 'MktABC_3',
       'Gamma_3', 'Manufacturer_3', 'Business_3', 'PRODUCT_STATUS_3',
       'ORIGINAL_SUPPLIER_3', 'SUBRANGE_3', 'OrderQty_3', 'OrderQty_4',
       'OrderQty_5', 'OrderQty_6', 'OrderQty_12', 'OrderQty_3_S',
       'OrderQty_4_S', 'OrderQty_5_S', 'OrderQty_6_S', 'OrderQty_12_S',
       'OrderQty_3_M', 'OrderQty_4_M', 'OrderQty_5_M', 'OrderQty_6_M',
       'OrderQty_12_M', 'OrderQty_fr', 'OrderQty_fr_12', 'OrderQty_change',
       'Mes', 'Mes_cat', 'SafetyStk_3_3_MESES', 'MOQ_3_3_MESES',
       'ROP_3_3_MESES', 'Volume_3_3_MESES', 'Length_3_3

In [78]:
categorical=["SalOrg","Mes_cat"]

categorical.extend(["DC_3","LogABC_3","MktABC_3","Gamma_3","Manufacturer_3","Business_3","PRODUCT_STATUS_3","ORIGINAL_SUPPLIER_3","SUBRANGE_3"])

no_contar=["Material","ID","date","Month",'OrderQty',"Month_ord"]

features=[x for x in train.columns if x not in no_contar]

features=[x for x in features if x not in categorical]

target=['OrderQty']

In [79]:
categorical

['SalOrg',
 'Mes_cat',
 'DC_3',
 'LogABC_3',
 'MktABC_3',
 'Gamma_3',
 'Manufacturer_3',
 'Business_3',
 'PRODUCT_STATUS_3',
 'ORIGINAL_SUPPLIER_3',
 'SUBRANGE_3']

In [80]:







for x in categorical:
    train[x+"_md"]=train[x]
    
categorical_md=[x+"_md" for x in categorical]


for l in categorical_md:
    le = preprocessing.LabelEncoder()
    le.fit(list(train[l].dropna()))
    train.loc[~train[l].isnull(),l]=le.transform(train.loc[~train[l].isnull(),l])

features.extend(categorical_md)

In [83]:
c

['SafetyStk',
 'LT',
 'MOQ',
 'ROP',
 'Age_ZN_ZI_years',
 'Comp_reference_number',
 'Name_Of_Competitor',
 'PRICE',
 'NEAREST_COMP_PRICE_MIN',
 'NEAREST_COMP_PRICE_MAX',
 'Gross_Weight',
 'Length',
 'Width',
 'Height',
 'Volume',
 'COMP_PRICE_MIN',
 'COMP_PRICE_AVG',
 'COMP_PRICE_MAX',
 'COMP_PRICE_MIN',
 'DC',
 'LogABC',
 'MktABC',
 'Gamma',
 'Manufacturer',
 'Business',
 'PRODUCT_STATUS',
 'ORIGINAL_SUPPLIER',
 'SUBRANGE']

In [84]:
categorical_indexes=[features.index(x) for x in categorical_md]

In [85]:
len(train.loc[train["Month"]<"2017-01"].reset_index(drop=True)[features].values)

3524174

In [86]:
len(train.loc[train["Month"].isin(["2017-01","2017-02","2017-03"])])

176040

In [87]:
train['OrderQty'].describe()

count    3.816242e+06
mean     1.099320e+01
std      1.162952e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.354000e+04
Name: OrderQty, dtype: float64

In [88]:
train['OrderQty'].quantile([0.9,0.95,0.99,0.999,0.9999])

0.9000       6.000
0.9500      20.000
0.9900     226.000
0.9990    1470.000
0.9999    4753.759
Name: OrderQty, dtype: float64

In [89]:
import time

start_time = time.clock()

#x_train = train.loc[train["Month"]<"2017-01"].reset_index(drop=True)[features].values
#y_train =train.loc[train["Month"]<"2017-01"].reset_index(drop=True)[target].values.ravel()

x_train = train.loc[train["Month"]<"2017-01"].reset_index(drop=True)[features].values
y_train =train.loc[train["Month"]<"2017-01"].reset_index(drop=True)[target].values.ravel()

x_test = train.loc[train["Month"].isin(["2017-01","2017-02","2017-03"])].reset_index(drop=True)[features].values
y_test =  train.loc[train["Month"].isin(["2017-01","2017-02","2017-03"])].reset_index(drop=True)[target].values.ravel()

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': { 'mae'},
        'num_leaves':250,
        'learning_rate': 0.01,
         "max_depth" : 8,
        'feature_fraction': 0.7,
         "bagging_freq": 1,
        'bagging_fraction': 0.5,
        'verbose': 1
}

lgbm3 = lgb.train(params,lgb_train,num_boost_round=800,valid_sets=lgb_eval,early_stopping_rounds=50,verbose_eval=5)


print(time.clock() - start_time, "seconds")

Training until validation scores don't improve for 50 rounds.
[5]	valid_0's l1: 19.5126
[10]	valid_0's l1: 18.663
[15]	valid_0's l1: 17.8516
[20]	valid_0's l1: 17.1021
[25]	valid_0's l1: 16.3985
[30]	valid_0's l1: 15.7284
[35]	valid_0's l1: 15.1087
[40]	valid_0's l1: 14.5421
[45]	valid_0's l1: 14.011
[50]	valid_0's l1: 13.5122
[55]	valid_0's l1: 13.0487
[60]	valid_0's l1: 12.6144
[65]	valid_0's l1: 12.218
[70]	valid_0's l1: 11.8523
[75]	valid_0's l1: 11.5199
[80]	valid_0's l1: 11.2142
[85]	valid_0's l1: 10.9223
[90]	valid_0's l1: 10.6586
[95]	valid_0's l1: 10.4088
[100]	valid_0's l1: 10.1828
[105]	valid_0's l1: 9.97475
[110]	valid_0's l1: 9.78165
[115]	valid_0's l1: 9.60706
[120]	valid_0's l1: 9.45056
[125]	valid_0's l1: 9.2997
[130]	valid_0's l1: 9.16566
[135]	valid_0's l1: 9.0375
[140]	valid_0's l1: 8.91012
[145]	valid_0's l1: 8.79972
[150]	valid_0's l1: 8.70284
[155]	valid_0's l1: 8.60991
[160]	valid_0's l1: 8.52317
[165]	valid_0's l1: 8.44254
[170]	valid_0's l1: 8.36028
[175]	valid

In [90]:
importancia=lgbm3.feature_importance(importance_type="gain")

def impxgb(valores,variables):
    dictimp={variables[a]:valores[a] for a in range(0,len(variables)) }
    xgimp=sorted(list(dictimp.items()), key=lambda x: x[1],reverse=True)

    return xgimp

ixg=impxgb(importancia,features)

ixg

### van a acumulados, "SafetyStk_3",'MOQ_3','ROP_3','Volume_3','Length_3','Width_3'


### canti

[('OrderQty_fr', 419449930873.78467),
 ('OrderQty_3', 118386740645.85458),
 ('OrderQty_3_S', 60308727678.737694),
 ('SafetyStk_3', 38758580182.671738),
 ('Mes', 37756885300.255249),
 ('OrderQty_12', 36331452560.812225),
 ('CBO_CBO_Qty_Shortage', 26900839497.007378),
 ('OrderQty_4', 24508699017.764061),
 ('OrderQty_12_M', 24168319053.27039),
 ('SafetyStk_3_3_MESES', 21111680405.866123),
 ('OrderQty_4_M', 20387652583.514133),
 ('OrderQty_fr_12', 16984086678.483627),
 ('OrderQty_3_M', 16847551897.194489),
 ('OrderQty_4_S', 12296588709.823393),
 ('OrderQty_5', 10085067238.771053),
 ('OrderQty_6', 9649133984.5095387),
 ('OrderQty_5_M', 9488143016.2240448),
 ('Age_ZN_ZI_years_3', 9041665636.3623466),
 ('OrderQty_6_M', 9008814210.4469223),
 ('OrderQty_12_S', 8521764622.1618519),
 ('MOQ_3', 8339247255.6495838),
 ('PRICE_3', 8138052992.6742363),
 ('OrderQty_5_S', 7662119210.3776789),
 ('NEAREST_COMP_PRICE_MIN_3', 7373073258.3422155),
 ('Length_3', 6595649951.8617802),
 ('LT_3', 6241285316.27995

In [91]:
t=train.loc[train["Month"].isin(["2017-04","2017-05","2017-06"]),]

In [92]:
t.reset_index(drop=True,inplace=True)

In [93]:
lgbm3.predict(t.loc[:,features].values,num_iteration=lgbm3.best_iteration)

array([ 6.24249893,  9.43971072,  4.64655364, ...,  1.75719769,
        1.19075315,  0.65501623])

In [94]:
t.loc[:,"demand"]=lgbm3.predict(t.loc[:,features].values,num_iteration=lgbm3.best_iteration)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [95]:
t["demand"].isnull().sum()

0

In [96]:
t["demand"].describe()

count    116028.000000
mean         17.298252
std         128.522889
min         -12.535393
25%           0.346248
50%           0.358867
75%           2.472308
max        6810.200121
Name: demand, dtype: float64

In [97]:

evaluation=evaluation.merge(t,on=['SalOrg',"Material","Month"],how="left")

In [98]:
evaluation["demand"].describe()

count    116028.000000
mean         17.298252
std         128.522889
min         -12.535393
25%           0.346248
50%           0.358867
75%           2.472308
max        6810.200121
Name: demand, dtype: float64

In [99]:
evaluation.loc[evaluation["demand"]<0,"demand"]=0

In [100]:
evaluation["ID"]=evaluation["ID_x"]

In [101]:
evaluation[["ID","demand"]].to_csv("submission25-corrected-2.csv",index=False)